#### Build to Google Drive <font color="DarkSeaGreen">/ Dr_UT_GROMACS_COLAB</font> [<img src="https://github.githubassets.com/favicons/favicon.png" width="16">](https://github.com/SampleBias/Dr_UT_GROMACS_COLAB)
<small>Suite: `Build_to_Google_Drive.ipynb` | [`GROMACS_for_CHARMM-GUI.ipynb`](https://colab.research.google.com/github/SampleBias/Dr_UT_GROMACS_COLAB/blob/main/GROMACS_for_CHARMM-GUI.ipynb) | [`GROMACS_for_production.ipynb`](https://colab.research.google.com/github/SampleBias/Dr_UT_GROMACS_COLAB/blob/main/GROMACS_for_production.ipynb) | [`Trajectory_analysis_tools.ipynb`](https://colab.research.google.com/github/SampleBias/Dr_UT_GROMACS_COLAB/blob/main/Trajectory_analysis_tools.ipynb)</small>

#### Documentation
**Before using this notebook, please click the *↳ cells hidden* button below to show the documentation.**

##### License

> This notebook as a work of software is licensed under the terms of the [AGPL-3.0](https://opensource.org/licenses/AGPL-3.0) or later.

##### About this software

> This notebook compiles and installs **GROMACS** and other libraries and utilities for running molecular dynamics simulations. It then caches the installations in your connected **Google Drive**, for later use in other notebooks.
>
> <font color="maroon">$\lower{0.25ex}{\smash{\LARGE ⚠}}$ To ensure compatiblity with every runtime type, please run this notebook on a "CPU only / Standard RAM" instance.</font>

#### Installation

**Run this installation by clicking *Runtime -> Run all* in the toolbar.** 

In [ ]:
#@markdown In the following cells, applications will be downloaded from the internet and compiled and/or installed to a **persistent cache** in your Google Drive.
#@markdown
#@markdown This cell sets up the cache folder.

import os

DRIVE_ROOT = "/content/drive/MyDrive"
REPO_NAME = "Dr_UT_GROMACS_COLAB"
STORAGE_SUBDIR = REPO_NAME

def drive_writable(root):
  if not os.path.isdir(root):
    return False
  try:
    probe = os.path.join(root, ".gdrive_probe")
    with open(probe, "w") as f:
      f.write("ok")
    os.remove(probe)
    return True
  except OSError:
    return False

def ensure_drive():
  if drive_writable(DRIVE_ROOT):
    return True
  try:
    from google.colab import drive  # type: ignore
  except ModuleNotFoundError:
    raise RuntimeError("Google Drive not mounted and google.colab is unavailable in this environment.")
  try:
    drive.mount("/content/drive", force_remount=False, timeout_ms=300000)
  except ValueError as e:
    if "mount failed" not in str(e):
      raise
    print("Initial mount failed, attempting force remount...")
    drive.mount("/content/drive", force_remount=True, timeout_ms=300000)
  return drive_writable(DRIVE_ROOT)

if not ensure_drive():
  raise RuntimeError(f"Drive mount point {DRIVE_ROOT} is not accessible. Please re-run and complete the authorization flow.")

storage = os.path.join(DRIVE_ROOT, STORAGE_SUBDIR)
os.makedirs(storage, exist_ok=True)
%env STORAGE={storage}

start = os.environ.get("START") or os.getcwd()
%env START={start}
%cd {start}

print(f"Using drive root: {DRIVE_ROOT}")
print(f"Storage dir: {storage}")


In [ ]:
%%bash
#@markdown **GROMACS** is downloaded prebuilt from [this notebook's GitHub repository](https://github.com/SampleBias/Dr_UT_GROMACS_COLAB).
#@markdown
#@markdown If not available, it is instead compiled from source code. (This takes a while.)

if [[ -d "/usr/local/gromacs" ]]; then
  exit 0  # already installed
fi

gromacs_vers="2025.4" #@param {type: "string"}
cache_gromacs="${STORAGE}/gromacs-${gromacs_vers}.tar.gz"

if [[ -s "${cache_gromacs}" ]]; then
  tar -xzf "${cache_gromacs}" -C "/usr/local"
else
  # Try to get a prebuilt archive...
  wget -q "https://raw.githubusercontent.com/SampleBias/Dr_UT_GROMACS_COLAB/main/prebuilt/gromacs-${gromacs_vers}.tar.gz"
  if [[ -s "gromacs-${gromacs_vers}.tar.gz" ]]; then
    tar -xzf "gromacs-${gromacs_vers}.tar.gz" -C "/usr/local"

    # Cache
    cp "gromacs-${gromacs_vers}.tar.gz" "${cache_gromacs}"

  # Prebuilt archive not available, so download the source code and build it...
  else
    wget -q "ftp://ftp.gromacs.org/gromacs/gromacs-${gromacs_vers}.tar.gz"
    if [[ ! -s "gromacs-${gromacs_vers}.tar.gz" ]]; then
      echo "Error: could not download: gromacs-${gromacs_vers}.tar.gz" >&2
      exit 1
    fi
    tar -xzf "gromacs-${gromacs_vers}.tar.gz"
    rm "gromacs-${gromacs_vers}.tar.gz"

    cd "gromacs-${gromacs_vers}"
    mkdir "build"
    cd "build"
    cmake .. -DGMX_BUILD_OWN_FFTW=ON -DGMX_GPU=CUDA
    make -j $(nproc)
    make install # -> /usr/local/gromacs

    # Cache
    tar -czf "my_gromacs.tar.gz" -C "/usr/local" "gromacs"
    mv "my_gromacs.tar.gz" "${cache_gromacs}"
  fi
fi

In [ ]:
%%bash
#@markdown Set up a Miniconda environment with **Python** `<=3.8`, **Numpy** `<=1.23`, and **NetworkX** `=2.3`, for `cgenff_charmm2gmx.py`; and a second environment for **Biopython** and **Open Babel**.

if [[ -d "${START}/miniconda3" ]]; then
  exit 0  # already installed
fi

miniconda3_vers="py310_23.5.2-0" #@param {type: "string"}

cache_miniconda3_installer="${STORAGE}/Miniconda3-${miniconda3_vers}-Linux-x86_64.sh.tar.gz"

if [[ -s "${cache_miniconda3_installer}" ]]; then
  tar -xzf "${cache_miniconda3_installer}"
else
  wget -q "https://repo.anaconda.com/miniconda/Miniconda3-${miniconda3_vers}-Linux-x86_64.sh"
  if [[ ! -s "Miniconda3-${miniconda3_vers}-Linux-x86_64.sh" ]]; then
    echo "Error: could not download: Miniconda3-${miniconda3_vers}-Linux-x86_64.sh" >&2
    exit 1
  fi

  tar -czf "my_miniconda3_installer.tar.gz" "Miniconda3-${miniconda3_vers}-Linux-x86_64.sh"
  mv "my_miniconda3_installer.tar.gz" "${cache_miniconda3_installer}"
fi

bash "Miniconda3-${miniconda3_vers}-Linux-x86_64.sh" -b -p "${START}/miniconda3"
rm "Miniconda3-${miniconda3_vers}-Linux-x86_64.sh"

eval "$("$START/miniconda3/bin/conda" shell.bash hook)"

cache_miniconda3="${STORAGE}/Miniconda3-${miniconda3_vers}-Linux-x86_64_envs.tar.gz"

if [[ -s "${cache_miniconda3}" ]]; then
  tar -xzf "${cache_miniconda3}" -C "${START}/miniconda3"
else
  conda create --yes --name "charmm2gmx"
  conda activate "charmm2gmx"
  conda config --env --set channel_priority "strict"
  conda config --env --add channels "conda-forge"
  conda install -q --yes "python<=3.8" "networkx=2.3" "numpy<=1.23"

  conda create --yes --name "biopython"
  conda activate "biopython"
  conda config --env --set channel_priority "strict"
  conda config --env --add channels "conda-forge"
  conda install -q --yes "biopython" "openbabel"

  # Cache
  tar -czf "my_miniconda3_envs.tar.gz" -C "${START}/miniconda3" "envs"
  mv "my_miniconda3_envs.tar.gz" "${cache_miniconda3}"
fi

In [ ]:
%%bash
#@markdown The CHARMM36 forcefield is downloaded.

if [[ -d "${START}/charmm36.ff" ]]; then
  exit 0  # already installed
fi

charmm36_vers="jul2022" #@param {type: "string"}
cache_charmm36="${STORAGE}/charmm36-${charmm36_vers}.tar.gz"

if [[ -s "${cache_charmm36}" ]]; then
  tar -xzf "${cache_charmm36}" -C "${START}"
else
  wget -q -O "charmm36-${charmm36_vers}.ff.tgz" "https://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/charmm36-${charmm36_vers}.ff.tgz"
  if [[ ! -s "charmm36-${charmm36_vers}.ff.tgz" ]]; then
    echo "Error: could not download: charmm36-${charmm36_vers}.ff.tgz" >&2
    exit 1
  fi
  tar -xzf "charmm36-${charmm36_vers}.ff.tgz"
  rm "charmm36-${charmm36_vers}.ff.tgz"

  mv "charmm36-${charmm36_vers}.ff" "${START}/charmm36.ff"

  # Cache
  tar -czf "my_charmm36.tar.gz" -C "${START}" "charmm36.ff"
  mv "my_charmm36.tar.gz" "${cache_charmm36}"
fi

In [ ]:
%%bash
#@markdown The utility **`cgenff_charmm2gmx.py`** is installed.

if [[ -x "${START}/miniconda3/envs/charmm2gmx/bin/cgenff_charmm2gmx.py" ]]; then
  exit 0  # already installed
fi

charmm2gmx_vers="py3_nx2" #@param {type: "string"}
cache_charmm2gmx="$STORAGE/cgenff_charmm2gmx_${charmm2gmx_vers}.tar.gz"

if [[ -s "${cache_charmm2gmx}" ]]; then
  tar -xzf "${cache_charmm2gmx}" -C "${START}/miniconda3/envs/charmm2gmx/bin"
else
  wget -q -O "cgenff_charmm2gmx.py" "https://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/cgenff_charmm2gmx_${charmm2gmx_vers}.py"
  if [[ ! -s "cgenff_charmm2gmx.py" ]]; then
    echo "Error: could not download: cgenff_charmm2gmx_${charmm2gmx_vers}.py" >&2
    exit 1
  fi
  sed -i"" 's/\r//g' "cgenff_charmm2gmx.py"
  chmod +x "cgenff_charmm2gmx.py"
  mv "cgenff_charmm2gmx.py" "${START}/miniconda3/envs/charmm2gmx/bin/"

  # Cache
  tar -czf "my_charmm2gmx.tar.gz" -C "${START}/miniconda3/envs/charmm2gmx/bin" "cgenff_charmm2gmx.py"
  mv "my_charmm2gmx.tar.gz" "${cache_charmm2gmx}"
fi

In [ ]:
#@markdown Finally, disconnect the runtime.
disconnect = True #@param {type: "boolean"}

if disconnect:
  from google.colab import drive, runtime
  drive.flush_and_unmount()
  runtime.unassign()